In [1]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

# Cargar modelo de embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')



/home/gugui/.local/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [5]:
import json

# Cargar el archivo JSON
with open("grafo_hierarquia.json", "r", encoding="utf-8") as file:
    grafo_json = json.load(file)
    
# Ahora `grafo_json` debería estar disponible para su uso en el bucle
nodos = []
for categoria, data in grafo_json.items():
    nodos.append(categoria)
    if 'subcategorias' in data:
        for subcategoria in data['subcategorias']:
            nodos.append(subcategoria)


In [6]:
# Generar los embeddings para los nodos
embeddings = model.encode(nodos)

In [7]:
# Crear índice FAISS
index = faiss.IndexFlatL2(embeddings.shape[1])  # Usar L2 distance
index.add(np.array(embeddings))

In [33]:
# Función de consulta
def consulta_rag(consulta, index, nodos, model, k=5):
    consulta_embedding = model.encode([consulta])
    D, I = index.search(np.array(consulta_embedding), k=k)  # Buscar los 'k' nodos más cercanos
    
    print(f"Consulta: {consulta}")
    for i in range(k):
        print(f"Resultado {i + 1}: {nodos[I[0][i]]} (Distancia: {D[0][i]:.4f})")


In [34]:
# Crear una lista de nodos (incluyendo plagas y momentos de aplicación)
nodos = []

# Recorrer la jerarquía de datos (categorías -> productos -> plagas -> momentos de aplicación)
for cultivo, datos_cultivo in grafo_json.items():
    nodos.append(cultivo)  # Añadir el cultivo como nodo
    if 'Productos' in datos_cultivo:
        for producto, info_producto in datos_cultivo['Productos'].items():
            nodos.append(producto)  # Añadir el producto como nodo
            if 'Controla a' in info_producto:
                for plaga, detalles in info_producto['Controla a'].items():
                    nodos.append(plaga)  # Añadir la plaga como nodo
                    for momento in detalles['Momento de aplicacion']:
                        nodos.append(momento)  # Añadir el momento de aplicación como nodo


In [ ]:
#Esta corrida demora aproximadamente 12 minutos

import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

# Crear un modelo de embeddings (por ejemplo, Sentence-BERT)
model = SentenceTransformer('all-MiniLM-L6-v2')

# Convertir los nodos a embeddings
nodos_embeddings = model.encode(nodos)

# Convertir los embeddings a un array NumPy
nodos_embeddings = np.array(nodos_embeddings)

# Guardar los embeddings como archivo .npy
np.save('nodos_embeddings.npy', nodos_embeddings)

# Crear un índice FAISS (usando el índice de búsqueda más cercano)
index = faiss.IndexFlatL2(nodos_embeddings.shape[1])  # Usar L2 para medir la distancia
index.add(nodos_embeddings)  # Agregar los embeddings al índice

# Guardar el índice FAISS en un archivo
faiss.write_index(index, 'nodos_index.faiss')

print("Embeddings y FAISS guardados exitosamente.")


Embeddings y FAISS guardados exitosamente.


In [35]:
# Función de consulta
def consulta_rag(consulta, index, nodos, model, k=10):
    consulta_embedding = model.encode([consulta])  # Codificar la consulta
    D, I = index.search(np.array(consulta_embedding), k)  # Buscar los k nodos más cercanos
    
    print(f"Consulta: {consulta}")
    for i in range(k):
        print(f"Resultado {i + 1}: {nodos[I[0][i]]} (Distancia: {D[0][i]:.4f})")

# Ejemplo de consulta
consulta_rag("soja", index, nodos, model)


Consulta: soja
Resultado 1: Soja (Distancia: 0.0000)
Resultado 2: Sorgo (Distancia: 0.8770)
Resultado 3: Arveja (Distancia: 0.8776)
Resultado 4: Cercospora sojina (Distancia: 0.9114)
Resultado 5: Cercospora sojina (Distancia: 0.9114)
Resultado 6: Cercospora sojina (Distancia: 0.9114)
Resultado 7: Cercospora sojina (Distancia: 0.9114)
Resultado 8: Cercospora sojina (Distancia: 0.9114)
Resultado 9: Cercospora sojina (Distancia: 0.9114)
Resultado 10: Cercospora sojina (Distancia: 0.9114)
